# Bayesian Estimation of Mixture Ratios in Mitochondrial DNA Samples (Refactored)

This notebook is a refactored version of the original pipeline. Implementation code lives in reusable Python modules under `src/mixture_bayes/`, and this notebook focuses on *configuration + calling the modules*.


In [ ]:
import os
import sys

# Ensure we can import from src/mixture_bayes when running from other working directories
_HERE = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
_SRC = os.path.abspath(_HERE)
if _SRC not in sys.path:
    sys.path.insert(0, _SRC)


In [ ]:
import numpy as np

from mixture_bayes.simulation import generate_informative_positions, simulate_and_estimate
from mixture_bayes.plotting import plot_posterior_distributions, plot_true_vs_estimated
from mixture_bayes.preprocessing import preprocess_variant_tables
from mixture_bayes.inference import bayesian_estimation


## Simulation-based validation

We simulate read counts at `M` informative positions under a known true mixture proportion `p_true`, then recover `p` via Bayesian inference on a grid.


In [ ]:
# Parameters
L = 16569          # Length of mitochondrial genome (human mtDNA)
M = 50             # Number of informative positions
mean_depth = 800   # Mean sequencing depth
epsilon = 0.02     # Sequencing error rate

rng = np.random.default_rng()
positions = generate_informative_positions(genome_length=L, n_positions=M, rng=rng)


In [ ]:
# Test mixture proportions
p_true_values = [0.001, 0.005, 0.01, 0.02]

results_list = []
for p_true in p_true_values:
    res = simulate_and_estimate(
        p_true=p_true,
        n_positions=M,
        mean_depth=mean_depth,
        epsilon=epsilon,
        positions=positions,
        rng=rng,
    )
    results_list.append(res)
    print(f'True proportion: {p_true*100:.2f}%')
    print(f"Estimated proportion (mean): {res['p_est_mean']*100:.4f}%")
    print(f"Estimated proportion (median): {res['p_est_median']*100:.4f}%")
    print(f"Estimated proportion (mode): {res['p_est_mode']*100:.4f}%")
    print(f"95% Credible Interval: [{res['lower_bound']*100:.4f}%, {res['upper_bound']*100:.4f}%]")
    print('-' * 50)


In [ ]:
plot_posterior_distributions(results_list, title='Posterior Distributions (Simulated Data)')
plot_true_vs_estimated(results_list, title='True vs Estimated Mixture Proportions (Simulated Data)')


## Sensitivity analysis (sequencing depth)

Hold `p_true` fixed and vary depth to observe uncertainty changes in the posterior and credible interval.


In [ ]:
depth_values = [100, 200, 400, 800]
p_true = 0.005
results_depth = []

for depth in depth_values:
    res = simulate_and_estimate(
        p_true=p_true,
        n_positions=M,
        mean_depth=depth,
        epsilon=epsilon,
        positions=positions,
        rng=rng,
    )
    results_depth.append(res)
    print(f'Sequencing Depth: {depth}x')
    print(f"Estimated p (mean): {res['p_est_mean']*100:.4f}%")
    print(f"95% Credible Interval: [{res['lower_bound']*100:.4f}%, {res['upper_bound']*100:.4f}%]")
    print('-' * 50)


In [ ]:
plot_posterior_distributions(results_depth, title='Posterior Distributions (Depth Sensitivity)')


## Preprocess `mutserve` variant tables (R, D, RDM)

This step extracts donor-informative positions from variant caller output and constructs `(k_reads, n_reads)` for Bayesian estimation.


In [ ]:
# Replace with your actual file paths
recipient_file = '/Users/hhakimjavadi/UFL Dropbox/Hesamedin Hakimjavadi/mixture_dna/workflow/mutserv/out/samples_variant_call_results/DA01_Batch1.txt'
donor_file =     '/Users/hhakimjavadi/UFL Dropbox/Hesamedin Hakimjavadi/mixture_dna/workflow/mutserv/out/samples_variant_call_results/DA02_Batch2.txt'
mixed_file =     '/Users/hhakimjavadi/UFL Dropbox/Hesamedin Hakimjavadi/mixture_dna/workflow/mutserv/out/samples_variant_call_results/RD_mix_0.01_2k.txt'

informative_df, common_variants_df = preprocess_variant_tables(
    recipient_file=recipient_file,
    donor_file=donor_file,
    mixed_file=mixed_file,
    mean_depth=800,
    donor_homoplasmy_threshold=0.99,
    recipient_low_threshold=0.01,
)

informative_df.head()


In [ ]:
# Run Bayesian estimation on real (preprocessed) informative sites
res_real = bayesian_estimation(
    k_reads=informative_df['k_reads'].values,
    n_reads=informative_df['n_reads'].values,
    epsilon=epsilon,
    p_max=0.05,
    p_grid_size=4000,
)

print(f"Estimated proportion (mean): {res_real['p_est_mean']*100:.4f}%")
print(f"Estimated proportion (median): {res_real['p_est_median']*100:.4f}%")
print(f"Estimated proportion (mode): {res_real['p_est_mode']*100:.4f}%")
print(f"95% Credible Interval: [{res_real['lower_bound']*100:.4f}%, {res_real['upper_bound']*100:.4f}%]")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(res_real['p_grid'] * 100, res_real['posterior'])
plt.xlabel('Mixture Proportion p (%)')
plt.ylabel('Posterior Probability')
plt.title('Posterior Distribution (Real Data)')
plt.grid(True)
plt.tight_layout()
plt.show()
